In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ogw3-loaded/data_frames_ogw3.pkl
/kaggle/input/ogw3-train-test/test_dataset.npz
/kaggle/input/ogw3-train-test/train_dataset.npz
/kaggle/input/ogw3-augmented/X_damage_augmented.npy
/kaggle/input/ogw3-augmented/X_freq_augmented.npy
/kaggle/input/ogw3-augmented/X_signals_augmented.npy


# 1. Data Loading and Preprocessing 

In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt


In [ ]:
%pwd

In [ ]:
os.chdir("..")

In [ ]:
%pwd

In [ ]:

%pwd

In [ ]:
os.chdir("input")

In [ ]:
for k, v in obj.items():
    print(k)
    print(v.shape)
    break

In [ ]:
len(obj)

In [ ]:
baseline_data={}
damage_data={}


In [ ]:
# Define the columns to extract (0-based indexing)
# columns_to_extract = [26, 34, 27, 35]  # Columns 27, 35, 28, 36 in 1-based indexing
columns_to_extract = [26]  # Columns 27 in 1-based indexing



In [ ]:
# Iterate through the items in the original dictionary
for k, v in obj.items():
    if 'baseline' in k:
        # Check if v is a Pandas DataFrame or NumPy array
        if isinstance(v, pd.DataFrame):
            filtered_data = v.iloc[:, columns_to_extract]  # Use .iloc for Pandas DataFrame
        else:
            filtered_data = v[:, columns_to_extract]  # Use NumPy slicing for arrays
        
        # Store the filtered data in the baseline dictionary
        baseline_data[k] = filtered_data
        
        # Print dimensions for baseline data
        #print(f"Baseline key: {k}, Data shape: {filtered_data.shape}")

    elif 'D1' in k:  # Filter for D1
        # Check if v is a Pandas DataFrame or NumPy array
        if isinstance(v, pd.DataFrame):
            filtered_data = v.iloc[:, columns_to_extract]  # Use .iloc for Pandas DataFrame
        else:
            filtered_data = v[:, columns_to_extract]  # Use NumPy slicing for arrays
        
        # Extract damage size from the key (number after 'S')
        # Split the key by underscores and find the part starting with 'S'
        damage_size_part = [part for part in k.split('_') if part.startswith('S')][0]
        damage_size = int(damage_size_part[1:])  # Extract the number after 'S'
        
        # Store the filtered data and damage size in the damage dictionary
        damage_data[k] = {
            'data': filtered_data,
            'damage_size': damage_size
        }
        
        # Print dimensions for damage data
        #print(f"Damage key: {k}, Data shape: {filtered_data.shape}, Damage size: {damage_size}")


In [ ]:
len(baseline_data)

In [ ]:
print(len(damage_data))

In [ ]:
# Initialize lists to store all samples and labels
X_all = []
y_all = []

# Add baseline samples
for k, v in baseline_data.items():
    X_all.append(v)  # Shape: (4369, 1)
    y_all.append(0)  # Label for baseline

# Add damage samples
for k, v in damage_data.items():
    X_all.append(v['data'])  # Shape: (4369, 1)
    y_all.append(v['damage_size'])  # Label for damage

# Convert lists to NumPy arrays
X_all = np.array(X_all)  # Shape: (num_samples, 4369, 4)
y_all = np.array(y_all)  # Shape: (num_samples,)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.2, random_state=42
)

# Print shapes for verification
print(f"Training data shape: {X_train.shape}")  # Should be (num_samples * 0.8, 4369, 4)
print(f"Training labels shape: {y_train.shape}")  # Should be (num_samples * 0.8,)
print(f"Test data shape: {X_test.shape}")  # Should be (num_samples * 0.2, 4369, 4)
print(f"Test labels shape: {y_test.shape}")  # Should be (num_samples * 0.2,)

In [ ]:
def print_label_distribution(y, set_name):
    """Print the distribution of labels in a dataset."""
    print(f"\n{set_name} label distribution:")
    print(f"- Baseline samples (0): {np.sum(y == 0)}")
    for damage_size in range(1, 14):
        print(f"- Damage size {damage_size}: {np.sum(y == damage_size)}")

In [ ]:
# Print label distribution for training and test sets
print_label_distribution(y_train, "Training set")
print_label_distribution(y_test, "Test set")

In [ ]:
%pwd

In [ ]:
os.chdir("..")
os.chdir("working")

In [ ]:
np.savez("train_dataset_D1.npz", X=X_train, y=y_train)
np.savez("test_dataset_D1.npz", X=X_test, y=y_test)

# 1.5) loading augmented data

In [2]:
%pwd


'/kaggle/working'

In [3]:
import os

In [4]:
os.chdir("..")
os.chdir("input")

In [5]:
X_signals_augmented = np.load('ogw3-augmented/X_signals_augmented.npy')
X_damage_augmented = np.load('ogw3-augmented/X_damage_augmented.npy')
X_freq_augmented = np.load('ogw3-augmented/X_freq_augmented.npy')


In [6]:
print(f"X_signals shape:{X_signals_augmented.shape}")
print(f"X_signals shape:{X_damage_augmented.shape}")
print(f"X_signals shape:{X_freq_augmented.shape}")

X_signals shape:(2040, 4369, 1)
X_signals shape:(2040, 14)
X_signals shape:(2040, 12)


In [7]:
import numpy as np

def summarize_data(X, name):
    print(f"Summary for {name}:")
    print("Mean:", np.mean(X, axis=0))
    print("Variance:", np.var(X, axis=0))
    print("Min:", np.min(X, axis=0))
    print("Max:", np.max(X, axis=0))
    print("-" * 50)

summarize_data(X_signals_augmented, "X_signals_augmented")
summarize_data(X_freq_augmented, "X_freq_augmented")


Summary for X_signals_augmented:
Mean: [[-0.00808563]
 [-0.03676917]
 [ 0.00793156]
 ...
 [ 0.43494834]
 [ 0.47826858]
 [ 0.47794448]]
Variance: [[0.04756828]
 [0.05728288]
 [0.05182533]
 ...
 [0.11266843]
 [0.11778735]
 [0.1192308 ]]
Min: [[-1.]
 [-1.]
 [-1.]
 ...
 [-1.]
 [-1.]
 [-1.]]
Max: [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
--------------------------------------------------
Summary for X_freq_augmented:
Mean: [0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333]
Variance: [0.07638889 0.07638889 0.07638889 0.07638889 0.07638889 0.07638889
 0.07638889 0.07638889 0.07638889 0.07638889 0.07638889 0.07638889]
Min: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Max: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
--------------------------------------------------


In [8]:
import pandas as pd

damage_counts = pd.Series(X_damage_augmented.flatten()).value_counts()
print("Damage Condition Counts:\n", damage_counts)


Damage Condition Counts:
 0.0    26520
1.0     2040
Name: count, dtype: int64


# 2. Execution and model Training
This is where the actual execution starts

In [9]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import os

In [10]:
os.chdir("..")
os.chdir("working")

In [11]:
os.chdir("..")
%pwd

'/kaggle'

In [12]:
os.chdir('input/ogw3-train-test')

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split

# ----------------------------------------------------------------------
# Step 1: Convert one-hot encoded damage labels to scalar values.
# ----------------------------------------------------------------------
# Each sample in X_damage_augmented is a one-hot vector. We convert it to the
# corresponding class index (i.e., 0 for undamaged, 1-13 for damage levels).
y_augmented = np.argmax(X_damage_augmented, axis=1).astype(np.float32)
# y_augmented now has shape (num_samples,)

In [14]:
y_augmented.min()

0.0

In [15]:
# ----------------------------------------------------------------------
# Step 2: Define the features.
# ----------------------------------------------------------------------
# For this regression task, we use the guided wave signals as the features.
# (If you wish to include frequency information as additional features, you might
# concatenate or process X_freq_augmented separately; here we assume only the signal is used.)
X_augmented = X_signals_augmented  # shape: (num_samples, 4369, 1)

In [16]:
# ----------------------------------------------------------------------
# Step 3: Split the data into training and test sets.
# ----------------------------------------------------------------------
# Here we use 80% of the data for training and 20% for testing.
X_train, X_test, y_train, y_test = train_test_split(
    X_augmented, y_augmented, test_size=0.1, random_state=42
)


In [17]:



# ----------------------------------------------------------------------
# Step 4: Verify the shapes of the datasets.
# ----------------------------------------------------------------------
print(f"Training data shape: {X_train.shape}")  # Expected: (num_train_samples, 4369, 1)
print(f"Training labels shape: {y_train.shape}")  # Expected: (num_train_samples,)
print(f"Test data shape: {X_test.shape}")         # Expected: (num_test_samples, 4369, 1)
print(f"Test labels shape: {y_test.shape}")         # Expected: (num_test_samples,)


Training data shape: (1836, 4369, 1)
Training labels shape: (1836,)
Test data shape: (204, 4369, 1)
Test labels shape: (204,)


In [18]:

# # Load training dataset
# train_data = np.load("train_dataset_D1.npz")
# X_train = train_data["X"]  # Load features
# y_train = train_data["y"]  # Load labels

# # Load test dataset
# test_data = np.load("test_dataset_D1.npz")
# X_test = test_data["X"]  # Load features
# y_test = test_data["y"]  # Load labels

# # Verify shapes
# print(f"Training data shape: {X_train.shape}")  # Should be (num_samples, 4369, 4)
# print(f"Training labels shape: {y_train.shape}")  # Should be (num_samples,)
# print(f"Test data shape: {X_test.shape}")  # Should be (num_samples, 4369, 4)
# print(f"Test labels shape: {y_test.shape}")  # Should be (num_samples,)

In [19]:
def print_label_distribution(y, set_name):
    """Print the distribution of labels in a dataset."""
    print(f"\n{set_name} label distribution:")
    print(f"- Baseline samples (0): {np.sum(y == 0)}")
    for damage_size in range(1, 14):
        print(f"- Damage size {damage_size}: {np.sum(y == damage_size)}")

In [20]:
# Print label distribution for training and test sets
print_label_distribution(y_train, "Training set")
print_label_distribution(y_test, "Test set")


Training set label distribution:
- Baseline samples (0): 433
- Damage size 1: 107
- Damage size 2: 107
- Damage size 3: 105
- Damage size 4: 106
- Damage size 5: 105
- Damage size 6: 107
- Damage size 7: 115
- Damage size 8: 109
- Damage size 9: 107
- Damage size 10: 111
- Damage size 11: 109
- Damage size 12: 111
- Damage size 13: 104

Test set label distribution:
- Baseline samples (0): 47
- Damage size 1: 13
- Damage size 2: 13
- Damage size 3: 15
- Damage size 4: 14
- Damage size 5: 15
- Damage size 6: 13
- Damage size 7: 5
- Damage size 8: 11
- Damage size 9: 13
- Damage size 10: 9
- Damage size 11: 11
- Damage size 12: 9
- Damage size 13: 16


In [21]:
# import numpy as np

# def min_max_scale(data):
#     """Normalize each feature (column) to [0, 1]."""
#     min_vals = np.min(data, keepdims=True)  # Min along time steps
#     max_vals = np.max(data, keepdims=True)  # Max along time steps
#     return (data - min_vals) / (max_vals - min_vals + 1e-8)  # Avoid division by zero


In [22]:
# X_train_scaled = np.array([min_max_scale(sample) for sample in X_train])
# X_test_scaled = np.array([min_max_scale(sample) for sample in X_test])


In [23]:
# X_train=X_train_scaled
# X_test=X_test_scaled

In [24]:
# Convert input data to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [40]:
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

In [26]:
print("Min values after scaling:", np.min(X_train, axis=(1,2)))
print("Max values after scaling:", np.max(X_train, axis=(1,2)))

Min values after scaling: [-1.040155  -0.9812768 -1.        ... -1.        -0.9722817 -0.9399686]
Max values after scaling: [1.0318567  0.94322634 1.         ... 1.         0.9312466  1.0010613 ]


In [27]:
print("X_train dtype:", X_train.dtype)  # Should print float32
print("y_train dtype:", y_train.dtype)  # Should print float32

X_train dtype: float32
y_train dtype: float32


In [28]:
import numpy as np

# Compute the minimum and maximum values from the training targets.
y_min = np.min(y_train)
y_max = np.max(y_train)

# Normalize the training and test targets.
y_train_norm = (y_train - y_min) / (y_max - y_min)
y_test_norm = (y_test - y_min) / (y_max - y_min)


print("Before Normalizing training targets (first 5):", y_train[:5])
print("Before Normalizing test targets (first 5):", y_test[:5])

print("Normalized training targets (first 5):", y_train_norm[:5])
print("Normalized test targets (first 5):", y_test_norm[:5])


Before Normalizing training targets (first 5): [ 5.  3. 13.  1. 10.]
Before Normalizing test targets (first 5): [7. 0. 6. 1. 8.]
Normalized training targets (first 5): [0.3846154  0.23076923 1.         0.07692308 0.7692308 ]
Normalized test targets (first 5): [0.53846157 0.         0.46153846 0.07692308 0.61538464]


# 3. Model Architecture

In [29]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [30]:
tf.keras.backend.set_floatx('float32')

In [31]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [32]:
# Multi-GPU setup
strategy = tf.distribute.MirroredStrategy()

In [33]:
# Transformer Encoder block definition 
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res


In [34]:
# Model building function adapted for multi-class classification
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
    num_classes=14  # Added parameter for number of classes
):
    inputs = keras.Input(shape=input_shape)
    # Reshape the input to 3D (batch_size, 874, 1)
    x = layers.Reshape((input_shape[0], 1))(inputs)
    
    # Apply multiple Transformer Encoder blocks
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    # Flatten the tensor for the MLP part
    x = layers.Flatten()(x)
    
    # MLP layers
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    
    # Output layer for multi-class classification
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


In [35]:
# Multi-GPU strategy scope
with strategy.scope():
    input_shape = (874,)  # Adjusted input shape

    model = build_model(
        input_shape,
        head_size=512,
        num_heads=8,
        ff_dim=128,
        num_transformer_blocks=4,
        mlp_units=[256],
        mlp_dropout=0.4,
        dropout=0.3,
        num_classes=14  # Specify 14 classes (0-13 damage levels)
    )

    # Compilation with multi-class classification loss
    # Use "sparse_categorical_crossentropy" if your labels are integer-encoded (0 to 13)
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        metrics=["accuracy"],
    )

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 874)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 874, 1)         │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 874, 1)         │         28,673 │ reshape[0][0],         │
│ (MultiHeadAttention)      │                        │                │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 874, 1)         │              0 │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 874, 1)         │              2 │ dropout_1[0][0]        │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 874, 1)         │              0 │ layer_normalization[0… │
│                           │                        │                │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 874, 128)       │            256 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 874, 128)       │              0 │ conv1d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 874, 1)         │            129 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 874, 1)         │              2 │ conv1d_1[0][0]         │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 874, 1)         │              0 │ layer_normalization_1… │
│                           │                        │                │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 874, 1)         │         28,673 │ add_1[0][0],           │
│ (MultiHeadAttention)      │                        │                │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 874, 1)         │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 874, 1)         │              2 │ dropout_4[0][0]        │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 874, 1)         │              0 │ layer_normalization_2… │
│                           │                        │                │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)    

 Total params: 343,846 (1.31 MB)

 Trainable params: 343,846 (1.31 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
X_train.shape

(1836, 4369, 1)

In [37]:
y_train.shape

(1836,)

In [41]:
y_train,y_test

(array([ 5,  3, 13, ...,  0,  4,  6], dtype=int32),
 array([ 7,  0,  6,  1,  8,  4,  0,  4,  0,  1,  4,  9,  9, 12,  3, 13,  5,
         5,  2,  9,  0,  0, 12,  8,  3,  0,  0,  4,  0,  1,  3,  0,  7,  5,
         5,  9, 10, 10,  4,  0,  0,  3,  5, 10, 13, 12,  7,  2,  0,  0,  0,
         0,  0, 11,  6,  6, 12,  0,  4,  0,  0,  3,  4,  6,  9,  0,  6,  8,
         5,  0,  6, 12,  0,  5,  3, 13,  1,  0,  9, 11,  6,  6,  9,  0,  8,
         4,  5,  0, 13, 12,  1,  2,  3,  0,  3,  0,  4,  8,  8,  5,  5,  3,
        12, 13, 13,  5,  0,  6,  0, 11, 11,  1,  0,  1,  8,  0,  0,  0, 13,
         0,  2, 12,  0, 11,  6, 13,  5, 10, 10,  4,  2, 13,  9,  2,  2,  4,
         0,  1,  9,  3,  0,  7, 11,  1,  0,  5,  8,  1,  9, 10,  5, 12,  1,
         0,  2,  3, 13,  5,  2,  0,  2,  4,  4,  0, 11,  0, 10, 13, 10,  4,
         3, 10,  9, 13,  2,  0,  8,  6,  3,  9,  6,  0,  6,  9, 11, 11,  3,
        13,  8,  0, 11, 13,  1,  8,  7,  2, 13, 13,  0,  3,  1, 11,  2,  0],
       dtype=int32))

In [42]:
# Assume X_train has the shape (num_samples, 4369, 1)
num_samples, original_length, channels = X_train.shape

# Calculate padding length so that the length becomes divisible by 5.
pad_len = 1

# Pad along the sequence axis (axis=1) with -np.inf so that the max operation is not affected.
X_train_padded = np.pad(
    X_train,
    pad_width=((0, 0), (0, pad_len), (0, 0)),
    mode='constant',
    constant_values=-np.inf
)

# Now X_train_padded has the shape (num_samples, 4370, 1).


In [43]:
X_train_padded.shape

(1836, 4370, 1)

In [44]:

# Reshape to group every 5 consecutive elements:
# New shape will be (num_samples, 874, 5, 1)
grouped_length = X_train_padded.shape[1] // 5  # Should be 874
X_train_grouped = X_train_padded.reshape(num_samples, grouped_length, 5, channels)

# Take the max over the 5 elements for each group (axis=2)
# Resulting shape is (num_samples, 874, 1)
X_train_max = np.max(X_train_grouped, axis=2)

# Optionally, if your model expects inputs of shape (num_samples, 874),
# reshape the array from (num_samples, 874, 1) to (num_samples, 874)
X_train_final = X_train_max.reshape(num_samples, grouped_length)

# Check the resulting shape
print("Shape after max-pooling and reshaping:", X_train_final.shape)


Shape after max-pooling and reshaping: (1836, 874)


In [45]:
# Assume X_train has the shape (num_samples, 4369, 1)
num_samples, original_length, channels = X_test.shape

# Calculate padding length so that the length becomes divisible by 5.
pad_len = 1

# Pad along the sequence axis (axis=1) with -np.inf so that the max operation is not affected.
X_test_padded = np.pad(
    X_test,
    pad_width=((0, 0), (0, pad_len), (0, 0)),
    mode='constant',
    constant_values=-np.inf
)

# Now X_train_padded has the shape (num_samples, 4370, 1).

# Reshape to group every 5 consecutive elements:
# New shape will be (num_samples, 874, 5, 1)
grouped_length = X_test_padded.shape[1] // 5  # Should be 874
X_test_grouped = X_test_padded.reshape(num_samples, grouped_length, 5, channels)

# Take the max over the 5 elements for each group (axis=2)
# Resulting shape is (num_samples, 874, 1)
X_test_max = np.max(X_test_grouped, axis=2)

# Optionally, if your model expects inputs of shape (num_samples, 874),
# reshape the array from (num_samples, 874, 1) to (num_samples, 874)
X_test_final = X_test_max.reshape(num_samples, grouped_length)

# Check the resulting shape
print("Shape after max-pooling and reshaping:", X_test_final.shape)


Shape after max-pooling and reshaping: (204, 874)


In [46]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback to save every epoch
checkpoint_callback = ModelCheckpoint(
    filepath='model_checkpoint.keras',  # Save file with epoch number and .keras extension
    save_weights_only=False,  # Save the full model
    save_freq='epoch',  # Save at the end of each epoch
    verbose=1  # To print a message when the model is saved
)

# Modify the callbacks list to include the checkpoint callback 
callbacks = [
    keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True),
    checkpoint_callback
]


In [48]:
%pwd

'/kaggle/input/ogw3-train-test'

In [49]:
os.chdir("..")
os.chdir("..")
os.chdir("working")

In [50]:
history = model.fit(
    X_train_final,
    y_train,
    validation_data=(X_test_final, y_test),
    epochs=200,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2347 - loss: 2.5360
Epoch 1: saving model to model_checkpoint.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.2344 - loss: 2.5370 - val_accuracy: 0.1569 - val_loss: 2.6258
Epoch 2/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2212 - loss: 2.5263
Epoch 2: saving model to model_checkpoint.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.2215 - loss: 2.5260 - val_accuracy: 0.1667 - val_loss: 2.6179
Epoch 3/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2290 - loss: 2.4968
Epoch 3: saving model to model_checkpoint.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.2292 - loss: 2.4969 - val_accuracy: 0.1667 - val_loss: 2.6044
Epoch 4/200
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2588 - loss: 2.4388
Epoch 4: saving model to model_checkpoint.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.2586 - loss: 2.4391 - val_accuracy: 0.1569 - val_loss: 2.5913
Epoch 5/200


# 4. Model Evaluation #

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Assume y_pred_norm are the predictions from your model (normalized)
y_pred_norm = model.predict(X_test).flatten()

# Convert normalized predictions back to the original scale.
y_pred = y_pred_norm * (y_max - y_min) + y_min

print("Predictions in the original scale (first 5):", y_pred[:5])


In [ ]:
# # Make predictions
# y_pred = model.predict(X_test).flatten()

In [ ]:
y_pred

In [ ]:
# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"""
Performance Metrics:
- MAE: {mae:.3f}
- MSE: {mse:.3f}
- RMSE: {rmse:.3f}
- R²: {r2:.3f}
""")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.plot([0,13], [0,13], 'r--')  # Perfect prediction line
plt.xlabel('True Damage Size')
plt.ylabel('Predicted Damage Size')
plt.title('True vs Predicted Values')
plt.show()

In [ ]:
residuals = y_test - y_pred

plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_pred, y=residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Analysis')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, bins=20)
plt.xlabel('Prediction Error')
plt.title('Error Distribution')
plt.show()

In [ ]:
error_by_size = []
for size in np.unique(y_test):
    mask = y_test == size
    if sum(mask) > 0:  # Check if size exists in test set
        size_mae = mean_absolute_error(y_test[mask], y_pred[mask])
        error_by_size.append((size, size_mae))

# Convert to DataFrame for better visualization
import pandas as pd
error_df = pd.DataFrame(error_by_size, columns=['Damage Size', 'MAE'])
print("\nMAE by Damage Size:")
print(error_df)

In [ ]:
# Separate baseline (label=0) and damage samples
baseline_mask = y_test == 0
damage_mask = y_test > 0

print(f"\nBaseline Performance (False Positives):")
print(f"- MAE: {mean_absolute_error(y_test[baseline_mask], y_pred[baseline_mask]):.3f}")

print(f"\nDamage Performance:")
print(f"- MAE: {mean_absolute_error(y_test[damage_mask], y_pred[damage_mask]):.3f}")

In [ ]:
# Find samples with largest errors
worst_indices = np.argsort(np.abs(residuals))[-5:]  # Top 5 worst predictions

print("\nWorst Predictions:")
for idx in worst_indices:
    print(f"True: {y_test[idx]:.1f}, Predicted: {y_pred[idx]:.2f}, Error: {residuals[idx]:.2f}")

# simple baseline cnn model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models


In [ ]:
def build_simple_cnn_model(input_shape=(4369, 1)):
    """
    A simple baseline CNN for regression.
    """
    inputs = layers.Input(shape=input_shape)
    
    # Block 1
    x = layers.Conv1D(32, kernel_size=7, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)  # Downsample by factor of 2
    
    # Block 2
    x = layers.Conv1D(64, kernel_size=5, activation='relu', padding='same')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)  # Downsample further
    
    # Block 3
    x = layers.Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    # Global average pooling aggregates over the time dimension.
    x = layers.GlobalAveragePooling1D()(x)
    
    # Fully-connected layers
    x = layers.Dense(128, activation='relu')(x)
    # x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='linear')(x)
    
    model = models.Model(inputs, outputs)
    return model


In [ ]:
# Build and compile the model.
model_simple = build_simple_cnn_model()
model_simple.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='mse',  # Mean Squared Error loss for regression
    metrics=['mae', 'mse']
)

model_simple.summary()

In [ ]:




# Train the model.
history_simple = model_simple.fit(
    X_train, y_train,  # Use your training data (ensure y_train is the target extent of damage)
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ]
)


In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(history_simple.history['loss'], label='Train Loss')
plt.plot(history_simple.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.title('Training vs. Validation Loss')
plt.show()


In [ ]:
# Use only a small subset of the training data
small_X_train = X_train[:20]
small_y_train =y_train[:20]

In [ ]:
small_y_train

In [ ]:


history_small = model_simple.fit(
    small_X_train, small_y_train,
    epochs=100,
    batch_size=8,
    verbose=1
)

# Evaluate on the small subset
small_loss = model_simple.evaluate(small_X_train, small_y_train)
print("Loss on small subset:", small_loss)


In [ ]:
# # Make predictions
y_pred_small = model_simple.predict(small_X_train).flatten()

In [ ]:
y_pred_small

try converting it again to a classification problem, and see if the model is able to learn any notable patterns